Import necessary libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
import xarray as xr
import numpy as np
import pandas as pd
import cdsapi

Change the directory where to download the data

In [ ]:
os.chdir('E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_1_hourly_box')
os.getcwd()

Download the data, it has been clipped to a box covering the USA and downloaded 6 years at a time, so there are 7 nc files in the directory, hourly rainfall values, units in meters 

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'variable': 'total_precipitation',
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],

        'area': [
            49.39, -124.74, 24.52,
            -66.95,
        ],
    },
    'US_box_ERA5_hourly_1981_1986.nc')

View the data in the downloaded directory

In [ ]:
# Specify the directory path where you want to search for .nc files
directory_path = "E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_1_hourly_box"

# List all files in the directory
file_list = os.listdir(directory_path)

# Filter the list to include only files ending with ".nc"
nc_files = [filename for filename in file_list if filename.endswith(".nc")]

# Print the filtered list of .nc files
for nc_file in nc_files:
    print(nc_file)

View one of the files

In [ ]:
data = xr.open_dataset('US_box_ERA5_hourly_2011_2016.nc')
data

In [ ]:
24 * 365* 4 + 24 * 366 *2

In [ ]:
del data

save each year as a separate NetCDF file (Hourly data), there will be total 42 nc files in the new directory, 42 files for each 42 years from 1981 - 2022

In [ ]:
# Specify the directory path where you want to search for .nc files
directory_path = "E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_1_hourly_box"

# List all files in the directory
file_list = os.listdir(directory_path)

# Filter the list to include only files with the ".nc" extension
nc_files = [filename for filename in file_list if filename.endswith(".nc")]

for nc_file in nc_files:
    file_path = os.path.join(directory_path, nc_file)
    data = xr.open_dataset(file_path)
    
    os.chdir('E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_2_hourly_year_separated_box')

    # Group the data by year
    grouped_data = data.groupby('time.year')

    # Iterate over the groups and save each year as a separate NetCDF file
    for year, group in grouped_data:
        year_data = group.copy()
        year_data.attrs = data.attrs  # Copy the global attributes

        # Create a new NetCDF file for the current year
        output_file = f'ERA5_hourly_box_{year}.nc'
        year_data.to_netcdf(output_file)

        print(f'Saved {output_file}')

    # Close the input NetCDF file
    data.close()

Open randomly for testing

In [ ]:
os.getcwd()

In [ ]:
data = xr.open_dataset('ERA5_hourly_box_1988.nc')
data

In [ ]:
24 * 366

In [ ]:
data = xr.open_dataset('ERA5_hourly_box_2011.nc')
data

In [ ]:
24 * 365

In [ ]:
del data

Aggregate hourly value to daily

In [19]:
# Specify the input directory containing the NetCDF files
input_directory = 'E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_2_hourly_year_separated_box'

# Specify the output directory where the new files will be saved
output_directory = 'E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_3_daily_year_separated_box'

# Get a list of all NetCDF files in the input directory
files = os.listdir(input_directory)
netcdf_files = [f for f in files if f.endswith('.nc')]

# Loop over each NetCDF file
for file in netcdf_files:
    # Create the input file path
    input_file = os.path.join(input_directory, file)

    # Open the NetCDF file
    ds = xr.open_dataset(input_file)

    # Extract the year from the file name
    year = file.split('_')[3].split('.')[0]

    # Resample the data from hourly to daily frequency and compute the sum
    ds_daily_sum = ds.tp.resample(time='D').sum()

    # Create the output file path
    output_file = os.path.join(output_directory, f"daily_rainfall_box_{year}.nc")

    # Save the aggregated data to a new NetCDF file
    ds_daily_sum.to_netcdf(output_file)

m to mm conversion

In [ ]:
# Specify the input directory containing the NetCDF files
input_directory = 'E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_3_daily_year_separated_box'

# Specify the output directory where the new files will be saved
output_directory = 'E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_4_daily_year_separated_box_mm'

# Get a list of all NetCDF files in the input directory
files = os.listdir(input_directory)
netcdf_files = [f for f in files if f.endswith('.nc')]

# Loop over each NetCDF file
for file in netcdf_files:
    # Create the input file path
    input_file = os.path.join(input_directory, file)

    # Open the NetCDF file
    ds = xr.open_dataset(input_file)

    # Extract the year from the file name
    year = file.split('_')[3].split('.')[0]

    # Convert 'tp' variable from meters to millimeters
    data_tp = ds.tp
    data_mm = data_tp * 1000

    # Create the output file path
    output_file = os.path.join(output_directory, f"daily_rainfall_mm_box_{year}.nc")

    # Save the converted data to a new NetCDF file
    data_mm.to_netcdf(output_file)

Combine all the nc files of the "_5_daily_ERA5_rainfall_mm_US_boundary" to a single netcdf called "ERA5_daily_rainfall_US_1981_2022.nc"

In [3]:
# Specify the directory path where your NetCDF files are located
directory_path = "E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_5_daily_ERA5_rainfall_mm_US_boundary"

# List all files in the directory
file_list = os.listdir(directory_path)

# Filter the list to include only files with the ".nc" extension
nc_files = [filename for filename in file_list if filename.endswith(".nc")]

datasets = []

# Loop through each NetCDF file and open it with Xarray
for nc_file in nc_files:
    file_path = os.path.join(directory_path, nc_file)
    dataset = xr.open_dataset(file_path)
    datasets.append(dataset)

# Concatenate the datasets along the time dimension
combined_dataset = xr.concat(datasets, dim="time")

# Close all datasets
for dataset in datasets:
    dataset.close()

# Specify the output file path for the combined dataset
combined_file_path = os.path.join(directory_path, "ERA5_daily_rainfall_US_1981_2022")

# Save the combined dataset to a new NetCDF file
combined_dataset.to_netcdf(combined_file_path)


Test the created combined data by opening it

In [5]:
os.chdir("E:\GEOG_7973_term_project\ERA5\ERA5_rainfall\_5_daily_ERA5_rainfall_mm_US_boundary")
os.getcwd()

'E:\\GEOG_7973_term_project\\ERA5\\ERA5_rainfall\\_5_daily_ERA5_rainfall_mm_US_boundary'

In [6]:
data = xr.open_dataset('ERA5_daily_rainfall_US_1981_2022')
data

<xarray.Dataset>
Dimensions:    (longitude: 232, latitude: 99, time: 15340)
Coordinates:
  * longitude  (longitude) float64 -124.7 -124.5 -124.2 ... -67.49 -67.24 -66.99
  * latitude   (latitude) float64 49.27 49.02 48.77 48.52 ... 25.27 25.02 24.77
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
Data variables:
    crs        (time) int32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-14)
    date:         2023-09-25 20:31:56

In [ ]:
del data